In [201]:
# import necessary libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
import numpy as np

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes
import folium

# !conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
request = requests.get(url)
data = request.text
soup = BeautifulSoup(data, "html.parser")

<h1>Set up the initial data frame </h1>

In [202]:
lst = soup.find_all('tr')

output = []
for row in lst:
    fullRow = []
    temp = row.find_all('td')
    for i in temp:
        if i.string!='/n':
            fullRow.append(i.get_text().rstrip())
    output.append(fullRow)
df = pd.DataFrame(output)
df = df[[0,1,2]]

In [203]:
df.columns = ['Postal Code','Borough','Neighborhood']
df = df[df.Borough != 'Not assigned']
df.dropna(subset=['Neighborhood','Postal Code'], inplace=True)
df.reset_index(drop=True,inplace=True)

In [204]:
df['Neighborhood'].replace('Not assigned', None, inplace=True)
df['Neighborhood'].fillna(df['Borough'],inplace=True)

In [205]:
df['Neighborhood'] = df.groupby(['Postal Code'])['Neighborhood'].transform(lambda x: ','.join(x))
df.drop_duplicates(inplace=True)
df.head(10)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,"Rouge,Malvern"
9,M3B,North York,Don Mills North
10,M4B,East York,"Woodbine Gardens,Parkview Hill"
12,M5B,Downtown Toronto,"Ryerson,Garden District"


In [206]:
df.shape

(106, 3)

<h1> Part 2: Add latitude and longitude data </h1>

In [207]:
# !wget -q -O 'Geo.csv' http://cocl.us/Geospatial_data
geo = pd.read_csv('http://cocl.us/Geospatial_data')
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [208]:
newDF = df.merge(geo, how='left', on='Postal Code')
newDF.shape

(106, 5)

In [209]:
newDF.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
5,M9A,Queen's Park,Queen's Park,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


In [342]:
# Create dataframe with only Boroughs from Toronto for analysis
df = newDF[newDF.Borough.str.contains('Toronto$', regex= True, na=False)]
df.reset_index(inplace=True,drop=True)
df.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
2,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.650571,-79.384568
9,M6H,West Toronto,"Dovercourt Village,Dufferin",43.669005,-79.442259


<h1>Part 3: Cluster and Analyze the Data </h1>

We are going to work with the Toronto area for our clustering

In [343]:
address = 'Toronto'

geolocator = Nominatim(user_agent="cn_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [344]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [346]:

VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + )
print('CLIENT_SECRET:' + )

SyntaxError: invalid syntax (<ipython-input-346-4e08ac8d0526>, line 5)

In [306]:
neighborhood_latitude = df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbourfront are 43.6542599, -79.3606359.


In [347]:
# type your answer here
limit = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_IDs, 
    CLIENT_SECRETs, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    limit)

url

NameError: name 'CLIENT_IDs' is not defined

In [308]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e1f629f60ba08001be485c0'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 46,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

In [309]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

In [310]:
import json
from pandas.io.json import json_normalize

In [311]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Gym / Fitness Center,43.653191,-79.357947
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980


In [312]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [313]:
toronto_venues = getNearbyVenues(names=df['Borough'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )


Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
East Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
West Toronto
Downtown Toronto
West Toronto
East Toronto
Downtown Toronto
West Toronto
East Toronto
Downtown Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
West Toronto
Central Toronto
Central Toronto
West Toronto
Central Toronto
Downtown Toronto
West Toronto
Central Toronto
Downtown Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
East Toronto


In [314]:
toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Downtown Toronto,43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Downtown Toronto,43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Downtown Toronto,43.654260,-79.360636,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center
3,Downtown Toronto,43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Downtown Toronto,43.654260,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
5,Downtown Toronto,43.654260,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
6,Downtown Toronto,43.654260,-79.360636,Dominion Pub and Kitchen,43.656919,-79.358967,Pub
7,Downtown Toronto,43.654260,-79.360636,Corktown Common,43.655618,-79.356211,Park
8,Downtown Toronto,43.654260,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site
9,Downtown Toronto,43.654260,-79.360636,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot


In [315]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Toronto,110,110,110,110,110,110
Downtown Toronto,1309,1309,1309,1309,1309,1309
East Toronto,125,125,125,125,125,125
West Toronto,166,166,166,166,166,166


In [316]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 235 uniques categories.


In [317]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [318]:
toronto_onehot.shape

(1710, 235)

In [319]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,Central Toronto,0.009091,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018182,...,0.000,0.009091,0.009091,0.000000,0.009091,0.000000,0.009091,0.000000,0.000000,0.000000
1,Downtown Toronto,0.002292,0.000764,0.000764,0.000764,0.000764,0.001528,0.002292,0.001528,0.010695,...,0.000,0.000764,0.000764,0.002292,0.012987,0.001528,0.005348,0.005348,0.000764,0.001528
2,East Toronto,0.024000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024000,...,0.008,0.000000,0.016000,0.000000,0.000000,0.000000,0.000000,0.008000,0.000000,0.000000
3,West Toronto,0.006024,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.000000,0.000000,0.000000,0.012048,0.000000,0.012048,0.006024,0.000000,0.000000


In [320]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.07
1  Sandwich Place  0.06
2            Café  0.05
3            Park  0.05
4             Gym  0.04


----Downtown Toronto----
                venue  freq
0         Coffee Shop  0.10
1                Café  0.05
2               Hotel  0.03
3          Restaurant  0.03
4  Italian Restaurant  0.02


----East Toronto----
                venue  freq
0    Greek Restaurant  0.07
1         Coffee Shop  0.06
2  Italian Restaurant  0.05
3      Ice Cream Shop  0.04
4             Brewery  0.04


----West Toronto----
                venue  freq
0                 Bar  0.08
1                Café  0.07
2         Coffee Shop  0.05
3          Restaurant  0.04
4  Italian Restaurant  0.04




In [321]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [322]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Park,Café,Sushi Restaurant,Dessert Shop,Gym,Pizza Place,Restaurant,Clothing Store
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Bakery,Japanese Restaurant,Italian Restaurant,Bar,Park,Steakhouse
2,East Toronto,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Brewery,Pizza Place,Café,Yoga Studio,Pub,American Restaurant
3,West Toronto,Bar,Café,Coffee Shop,Restaurant,Italian Restaurant,Pizza Place,Bakery,Park,Grocery Store,Mexican Restaurant


In [323]:
# set number of clusters.
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 2, 1], dtype=int32)

In [324]:
temp = df.drop(['Neighborhood','Postal Code'],axis=1)
temp = temp.rename(columns={'Borough':'Neighborhood'})
temp.head()

,Neighborhood,Latitude,Longitude
0,Downtown Toronto,43.654260,-79.360636
1,Downtown Toronto,43.662301,-79.389494
2,Downtown Toronto,43.657162,-79.378937
3,Downtown Toronto,43.651494,-79.375418
4,East Toronto,43.676357,-79.293031


In [325]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# df.drop('Postal Code', inplace=True)
toronto_merged = temp
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how='left')

toronto_merged.head(10) # check the last columns!

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,43.654260,-79.360636,1,Coffee Shop,Café,Restaurant,Hotel,Bakery,Japanese Restaurant,Italian Restaurant,Bar,Park,Steakhouse
1,Downtown Toronto,43.662301,-79.389494,1,Coffee Shop,Café,Restaurant,Hotel,Bakery,Japanese Restaurant,Italian Restaurant,Bar,Park,Steakhouse
2,Downtown Toronto,43.657162,-79.378937,1,Coffee Shop,Café,Restaurant,Hotel,Bakery,Japanese Restaurant,Italian Restaurant,Bar,Park,Steakhouse
3,Downtown Toronto,43.651494,-79.375418,1,Coffee Shop,Café,Restaurant,Hotel,Bakery,Japanese Restaurant,Italian Restaurant,Bar,Park,Steakhouse
4,East Toronto,43.676357,-79.293031,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Brewery,Pizza Place,Café,Yoga Studio,Pub,American Restaurant
5,Downtown Toronto,43.644771,-79.373306,1,Coffee Shop,Café,Restaurant,Hotel,Bakery,Japanese Restaurant,Italian Restaurant,Bar,Park,Steakhouse
6,Downtown Toronto,43.657952,-79.387383,1,Coffee Shop,Café,Restaurant,Hotel,Bakery,Japanese Restaurant,Italian Restaurant,Bar,Park,Steakhouse
7,Downtown Toronto,43.669542,-79.422564,1,Coffee Shop,Café,Restaurant,Hotel,Bakery,Japanese Restaurant,Italian Restaurant,Bar,Park,Steakhouse
8,Downtown Toronto,43.650571,-79.384568,1,Coffee Shop,Café,Restaurant,Hotel,Bakery,Japanese Restaurant,Italian Restaurant,Bar,Park,Steakhouse
9,West Toronto,43.669005,-79.442259,1,Bar,Café,Coffee Shop,Restaurant,Italian Restaurant,Pizza Place,Bakery,Park,Grocery Store,Mexican Restaurant


In [332]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [338]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,43.728020,-79.388790,0,Coffee Shop,Sandwich Place,Park,Café,Sushi Restaurant,Dessert Shop,Gym,Pizza Place,Restaurant,Clothing Store
19,Central Toronto,43.711695,-79.416936,0,Coffee Shop,Sandwich Place,Park,Café,Sushi Restaurant,Dessert Shop,Gym,Pizza Place,Restaurant,Clothing Store
20,Central Toronto,43.712751,-79.390197,0,Coffee Shop,Sandwich Place,Park,Café,Sushi Restaurant,Dessert Shop,Gym,Pizza Place,Restaurant,Clothing Store
21,Central Toronto,43.696948,-79.411307,0,Coffee Shop,Sandwich Place,Park,Café,Sushi Restaurant,Dessert Shop,Gym,Pizza Place,Restaurant,Clothing Store
23,Central Toronto,43.715383,-79.405678,0,Coffee Shop,Sandwich Place,Park,Café,Sushi Restaurant,Dessert Shop,Gym,Pizza Place,Restaurant,Clothing Store
24,Central Toronto,43.672710,-79.405678,0,Coffee Shop,Sandwich Place,Park,Café,Sushi Restaurant,Dessert Shop,Gym,Pizza Place,Restaurant,Clothing Store
26,Central Toronto,43.704324,-79.388790,0,Coffee Shop,Sandwich Place,Park,Café,Sushi Restaurant,Dessert Shop,Gym,Pizza Place,Restaurant,Clothing Store
29,Central Toronto,43.689574,-79.383160,0,Coffee Shop,Sandwich Place,Park,Café,Sushi Restaurant,Dessert Shop,Gym,Pizza Place,Restaurant,Clothing Store
31,Central Toronto,43.686412,-79.400049,0,Coffee Shop,Sandwich Place,Park,Café,Sushi Restaurant,Dessert Shop,Gym,Pizza Place,Restaurant,Clothing Store


In [339]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,43.654260,-79.360636,1,Coffee Shop,Café,Restaurant,Hotel,Bakery,Japanese Restaurant,Italian Restaurant,Bar,Park,Steakhouse
1,Downtown Toronto,43.662301,-79.389494,1,Coffee Shop,Café,Restaurant,Hotel,Bakery,Japanese Restaurant,Italian Restaurant,Bar,Park,Steakhouse
2,Downtown Toronto,43.657162,-79.378937,1,Coffee Shop,Café,Restaurant,Hotel,Bakery,Japanese Restaurant,Italian Restaurant,Bar,Park,Steakhouse
3,Downtown Toronto,43.651494,-79.375418,1,Coffee Shop,Café,Restaurant,Hotel,Bakery,Japanese Restaurant,Italian Restaurant,Bar,Park,Steakhouse
5,Downtown Toronto,43.644771,-79.373306,1,Coffee Shop,Café,Restaurant,Hotel,Bakery,Japanese Restaurant,Italian Restaurant,Bar,Park,Steakhouse
6,Downtown Toronto,43.657952,-79.387383,1,Coffee Shop,Café,Restaurant,Hotel,Bakery,Japanese Restaurant,Italian Restaurant,Bar,Park,Steakhouse
7,Downtown Toronto,43.669542,-79.422564,1,Coffee Shop,Café,Restaurant,Hotel,Bakery,Japanese Restaurant,Italian Restaurant,Bar,Park,Steakhouse
8,Downtown Toronto,43.650571,-79.384568,1,Coffee Shop,Café,Restaurant,Hotel,Bakery,Japanese Restaurant,Italian Restaurant,Bar,Park,Steakhouse
9,West Toronto,43.669005,-79.442259,1,Bar,Café,Coffee Shop,Restaurant,Italian Restaurant,Pizza Place,Bakery,Park,Grocery Store,Mexican Restaurant
10,Downtown Toronto,43.640816,-79.381752,1,Coffee Shop,Café,Restaurant,Hotel,Bakery,Japanese Restaurant,Italian Restaurant,Bar,Park,Steakhouse


In [340]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,43.676357,-79.293031,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Brewery,Pizza Place,Café,Yoga Studio,Pub,American Restaurant
12,East Toronto,43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Brewery,Pizza Place,Café,Yoga Studio,Pub,American Restaurant
15,East Toronto,43.668999,-79.315572,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Brewery,Pizza Place,Café,Yoga Studio,Pub,American Restaurant
17,East Toronto,43.659526,-79.340923,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Brewery,Pizza Place,Café,Yoga Studio,Pub,American Restaurant
38,East Toronto,43.662744,-79.321558,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Brewery,Pizza Place,Café,Yoga Studio,Pub,American Restaurant


In [348]:
# Complete